In [ ]:
import os
import pandas as pd

def launchComplex2coords(LC):
    print('LC=*%s*' % LC)
    if LC == '39B': # NASA Space Launch System
        lon = -80.620833
        lat = 28.627222
    elif LC == '39A': # SpaceX Falcon Heavy & Falcon 9
        lon = -80.604333
        lat = 28.608389  
    elif LC == '41': # ULA Atlas V       
        lon = -80.583056
        lat = 28.583333
    elif LC == '40': # SpaceX Falcon 9
        lat = 28.562106
        rlon = -80.57718
    elif LC == '37B': # ULA Delta IV Heavy and Medium
        lat = 28.531986
        lon = -80.566821   
    elif LC == '13': # Falcon landing site LZ-1 and LZ-2
        lat = 28.4859
        lon = -80.5444
    return lat, lon

# Get event parameters from file
event_params = pd.read_csv('event_params.csv', skipinitialspace=True)
event_params.columns = event_params.columns.str.lstrip()

event_params['fdsnclient']="RASPISHAKE"
event_params['network'] = 'AM'
event_params['seisandb'] = 'KSC__'
event_params['lat'] = None
event_params['lon'] = None
event_params['pretrigsecs'] = None
event_params['posttrigsecs'] = None
event_params['searchRadiusKm'] = 20.0 
print(event_params.columns)

# Complete event parameters
for i,row in event_params.iterrows():
    LC = row['launch_complex']
    lat, lon = launchComplex2coords(LC)
    event_params.loc[i, 'lat'] = lat
    event_params.loc[i, 'lon'] = lon    
    event_params.loc[i, 'pretrigsecs']=60.0
    event_params.loc[i, 'posttrigsecs']=120.0+event_params.loc[i, 'pretrigsecs']
    if row['bool_lander']:
        event_params.loc[i, 'posttrigsecs'] += 600.0 # capture the landing signal too

# Save to master event file
masterEventFile = 'fdsn_catalog.csv'
if os.path.exists(masterEventFile):
    df = pd.read_csv(masterEventFile)
    df = df.append(event_params)
else:
    df = event_params
df.drop_duplicates().to_csv(masterEventFile, index=False)